In [33]:
from __future__ import print_function
from __future__ import absolute_import
import matplotlib.pyplot as plt

import sugartensor as tf
import os
from IPython.display import display, Image
import numpy as np
from scipy import ndimage
from six.moves import cPickle as pickle

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
%matplotlib inline


In [34]:
batch_size = 40   # batch size
cat_dim = 5   # total categorical factor
con_dim = 11    # total continuous factor
rand_dim = 38  # total random latent dimension

In [35]:

fn = os.listdir("barcode")
len(fn)

10000

In [36]:
image_size = 36
pixel_depth = 255.0
# put all the images into this blob of size 10kx108x108x3
dataset = np.ndarray(shape = (10000, 36, 36, 1), dtype = np.float32)
counter = 0
training_label = np.ndarray(shape = (10000), dtype = np.int64)
for file in fn:
    image_data = (ndimage.imread("barcode/"+file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
    if image_data.shape != (36, 36, 3):
        print("wrong size")
#     print(file[4])
    training_label[counter] = int(file[4])-1
    dataset[counter, :, :] = image_data[:,:,0:1]
    counter+=1
training_image, training_label = tf.sg_data._data_to_tensor([dataset, training_label], batch_size, name = "train")

In [37]:
training_image

<tf.Tensor 'train_5:0' shape=(40, 36, 36, 1) dtype=float32>

In [38]:
training_label

<tf.Tensor 'train_5:1' shape=(40,) dtype=int64>

In [39]:
# generator network
def generator(tensor):

    # reuse flag
    reuse = len([t for t in tf.global_variables() if t.name.startswith('generator')]) > 0

    with tf.sg_context(name='generator', size=4, stride=2, act='relu', bn=True, reuse=reuse):
        res = (tensor
               .sg_dense(dim=1024, name='fc1')
               .sg_dense(dim=9*9*128, name='fc2')
               .sg_reshape(shape=(-1, 9, 9, 128))
               .sg_upconv(dim=64, name='conv1')
               .sg_upconv(dim=1, act='sigmoid', bn=False, name='conv2'))
    return res

In [40]:
def discriminator(tensor):

    # reuse flag
    reuse = len([t for t in tf.global_variables() if t.name.startswith('discriminator')]) > 0

    with tf.sg_context(name='discriminator', size=4, stride=2, act='leaky_relu', reuse=reuse):
        # shared part
        shared = (tensor
                  .sg_conv(dim=64, name='conv1')
                  .sg_conv(dim=128, name='conv2')
                  .sg_flatten()
                  .sg_dense(dim=1024, name='fc1'))

        # discriminator end
        disc = shared.sg_dense(dim=1, act='linear', name='disc').sg_squeeze()

        # shared recognizer part
        recog_shared = shared.sg_dense(dim=128, name='recog')

        # categorical auxiliary classifier end
        cat = recog_shared.sg_dense(dim=1, act='sigmoid', name='cat')

        # continuous auxiliary classifier end
        con = recog_shared.sg_dense(dim=con_dim, act='sigmoid', name='con')

        return disc, cat, con

In [41]:
#
# inputs
#

# input images and label
x = training_image
y = training_label

# labels for discriminator
y_real = tf.ones(batch_size)
y_fake = tf.zeros(batch_size)

# discriminator labels ( half 1s, half 0s )
y_disc = tf.concat([y, y * 0], 0)

# categorical latent variable
z_cat = tf.to_float(tf.multinomial(tf.ones((batch_size, cat_dim), dtype=tf.sg_floatx) / cat_dim, 1))
print(z_cat)
# continuous latent variable
z_con = tf.random_normal((batch_size, con_dim))
print(z_con)
# random latent variable dimension
z_rand = tf.random_normal((batch_size, rand_dim))
print(z_rand)
# latent variable
z = tf.concat([z_cat, z_con, z_rand], 1)

Tensor("ToFloat_2:0", shape=(40, 1), dtype=float32)
Tensor("random_normal_26:0", shape=(40, 11), dtype=float32)
Tensor("random_normal_27:0", shape=(40, 38), dtype=float32)


In [42]:
#
# Computational graph
#

# generator
gen = generator(z)
gen

ValueError: Trying to share variable generator/fc1/W, but specified shape (50, 1024) and found shape (41, 1024).

In [ ]:
# add image summary
tf.sg_summary_image(x, name='real')
tf.sg_summary_image(gen, name='fake')

# discriminator
disc_real, cat_real, _ = discriminator(x)
disc_fake, cat_fake, con_fake = discriminator(gen)

In [ ]:
gen

In [ ]:
#
# loss
#

# discriminator loss
loss_d_r = disc_real.sg_bce(target=y_real, name='disc_real')
loss_d_f = disc_fake.sg_bce(target=y_fake, name='disc_fake')
loss_d = (loss_d_r + loss_d_f) / 2

loss_d

# generator loss
loss_g = disc_fake.sg_bce(target=y_real, name='gen')

# categorical factor loss
loss_c_r = cat_real.sg_ce(target=y, name='cat_real')
loss_c_d = cat_fake.sg_ce(target=z_cat, name='cat_fake')
loss_c = (loss_c_r + loss_c_d) / 2

# continuous factor loss
loss_con = con_fake.sg_mse(target=z_con, name='con').sg_mean(dims=1)


#
# train ops
#

# discriminator train ops
train_disc = tf.sg_optim(loss_d + loss_c + loss_con, lr=0.00005, category='discriminator')
# generator train ops
train_gen = tf.sg_optim(loss_g + loss_c + loss_con, lr=0.0005, category='generator')


#
# training
#

# def alternate training func
@tf.sg_train_func
def alt_train(sess, opt):
    l_disc = sess.run([loss_d, train_disc])[0]  # training discriminator
    l_gen = sess.run([loss_g, train_gen])[0]  # training generator
    return np.mean(l_disc) + np.mean(l_gen)

# do training
alt_train(log_interval=60, max_ep=120, ep_size=10000//batch_size, early_stop=False, save_dir='asset/train/acgan-barcode9')

In [ ]:
sess.close()
